In [1]:
import xspec as x
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from astropy.io import fits



In [2]:

#Load relxill and test it

# --- STEP 1: DEFINE PATHS ---
# Path to the folder where you compiled relxill (contains lmodel.dat)
relxill_dir = "/home/kyle/software/relxill"

# Path to the folder containing the heavy FITS tables (often the same dir)
relxill_tables = "/home/kyle/software/relxill"

# --- STEP 2: SET ENVIRONMENT VARIABLE ---
# This tells relxill where to look for the tables so you don't have to copy them
os.environ["RELXILL_TABLE_PATH"] = relxill_tables

# --- STEP 3: LOAD THE MODEL ---
# This mimics the "lmod relxill /path" command
print("Loading Relxill...")
x.AllModels.lmod("relxill", relxill_dir)

# --- STEP 4: USE IT ---
# Now you can use 'relxill', 'relxillCp', 'relxilllp', etc.
# Example: simple relativistic reflection
m = x.Model("tbabs * relxillns")

# Verify it loaded
print(m.componentNames)

Loading Relxill...
Model package relxill successfully loaded.
['TBabs', 'relxillNS']
tbvabs Version 2.3
Cosmic absorption with grains and H2, modified from
Wilms, Allen, & McCray, 2000, ApJ 542, 914-924
Questions: Joern Wilms
joern.wilms@sternwarte.uni-erlangen.de
joern.wilms@fau.de

http://pulsar.sternwarte.uni-erlangen.de/wilms/research/tbabs/

PLEASE NOTICE:
To get the model described by the above paper
you will also have to set the abundances:
   abund wilm

Note that this routine ignores the current cross section setting
as it always HAS to use the Verner cross sections as a baseline.
 *** loading RELXILL model (version 2.5) *** 

Model TBabs<1>*relxillNS<2> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   relxillNS  Index1              3.00000      frozen
   3    2   relxillNS  Index2              3.00000      frozen
   4    2   relxillNS  Rbr              

In [3]:
# --- 1. SETUP & DATA LOADING ---
x.AllData.clear()
x.AllModels.clear()

# --- PN: interval + full-spectrum options ---
time_intervals_pn = {
    "Full": {
        "pn_spec": "../products/0865600201/pn/spec/pn_source_spectrum.fits",
        "pn_bkg":  "../products/0865600201/pn/spec/pn_bkg_spectrum.fits",
        "pn_rmf":  "../products/0865600201/pn/spec/pn_rmf.rmf",
        "pn_arf":  "../products/0865600201/pn/spec/pn_arf.arf",
    },
    "Full_grp": {
        "pn_spec": "../products/0865600201/pn/spec/pn_source_spectrum_grp.fits",
        "pn_bkg":  "../products/0865600201/pn/spec/pn_bkg_spectrum.fits",
        "pn_rmf":  "../products/0865600201/pn/spec/pn_rmf.rmf",
        "pn_arf":  "../products/0865600201/pn/spec/pn_arf.arf",
    },
    "Dipping": {
        "pn_spec": "../products/0865600201/pn/spec/pn_source_Dipping.fits",
        "pn_bkg":  "../products/0865600201/pn/spec/pn_bkg_Dipping.fits",
        "pn_rmf":  "../products/0865600201/pn/spec/pn_rmf_Dipping.rmf",
        "pn_arf":  "../products/0865600201/pn/spec/pn_arf_Dipping.arf",
    },
    "Dipping_grp": {
        "pn_spec": "../products/0865600201/pn/spec/pn_source_Dipping_grp.pha",
        "pn_bkg":  "../products/0865600201/pn/spec/pn_bkg_Dipping.fits",
        "pn_rmf":  "../products/0865600201/pn/spec/pn_rmf_Dipping.rmf",
        "pn_arf":  "../products/0865600201/pn/spec/pn_arf_Dipping.arf",
    },
    "Persistent": {
        "pn_spec": "../products/0865600201/pn/spec/pn_source_Persistent.fits",
        "pn_bkg":  "../products/0865600201/pn/spec/pn_bkg_Persistent.fits",
        "pn_rmf":  "../products/0865600201/pn/spec/pn_rmf_Persistent.rmf",
        "pn_arf":  "../products/0865600201/pn/spec/pn_arf_Persistent.arf",
    },
    "Persistent_grp": {
        "pn_spec": "../products/0865600201/pn/spec/pn_source_Persistent_grp.pha",
        "pn_bkg":  "../products/0865600201/pn/spec/pn_bkg_Persistent.fits",
        "pn_rmf":  "../products/0865600201/pn/spec/pn_rmf_Persistent.rmf",
        "pn_arf":  "../products/0865600201/pn/spec/pn_arf_Persistent.arf",
    },
    "Shallow": {
        "pn_spec": "../products/0865600201/pn/spec/pn_source_Shallow.fits",
        "pn_bkg":  "../products/0865600201/pn/spec/pn_bkg_Shallow.fits",
        "pn_rmf":  "../products/0865600201/pn/spec/pn_rmf_Shallow.rmf",
        "pn_arf":  "../products/0865600201/pn/spec/pn_arf_Shallow.arf",
    },
    "Shallow_grp": {
        "pn_spec": "../products/0865600201/pn/spec/pn_source_Shallow_grp.pha",
        "pn_bkg":  "../products/0865600201/pn/spec/pn_bkg_Shallow.fits",
        "pn_rmf":  "../products/0865600201/pn/spec/pn_rmf_Shallow.rmf",
        "pn_arf":  "../products/0865600201/pn/spec/pn_arf_Shallow.arf",
    },
}

In [4]:
# Choose: "Full", "Dipping", "Persistent", "Shallow"
selected_interval = "Full"
pn_data = time_intervals_pn[selected_interval]

pn_spec = pn_data["pn_spec"]
pn_bkg  = pn_data["pn_bkg"]
pn_rmf  = pn_data["pn_rmf"]
pn_arf  = pn_data["pn_arf"]

# Load PN (once)
x.AllData(pn_spec)
s = x.AllData(1)
s.background = pn_bkg
s.response = pn_rmf
s.response.arf = pn_arf

x.AllData.ignore("bad")
s.ignore("**-0.6 7.5-**")  # Ignore outside 0.8-10 keV

# Global XSPEC settings
x.Plot.device = "/null"
x.Plot.xAxis = "keV"
x.Xset.xsect = "vern"
x.Xset.abund = "wilm"


# --- 2. MODEL DEFINITION ---
# Defining the model for the loaded PN data
m1 = x.Model("tbabs * (nthcomp + diskbb + bbodyrad)")



1 spectrum  in use
 
Spectral Data File: ../products/0865600201/pn/spec/pn_source_spectrum.fits  Spectrum 1
Net count rate (cts/s) for Spectrum:1  4.220e+00 +/- 7.206e-03
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-4096
  Telescope: XMM Instrument: EPN  Channel Type: PI
  Exposure Time: 8.128e+04 sec
 Using fit statistic: chi
 No response loaded.

***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
Net count rate (cts/s) for Spectrum:1  5.948e-01 +/- 2.673e-02 (14.1 % total)
***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
Response successfully loaded.
Arf successfully loaded.

ignore:     0 channels ignored from  source number 1
   115 channels (1-115) ignored in spectrum #     1
  2597 channels (1500-4096) ignored in spectrum #     1

 Cross Section Table set to vern:  Verner, Ferland, Korista, and Yakovlev 1996
 Solar Abundance Vector set to wilm:  Wilms, J., A

In [5]:
# --- 4. FITTING ---



m1(1).values = 0.2
m1(1).frozen = False
m1(2).values = 1.7
m1(3).values = 10
m1(3).frozen = False
m1(4).link = "p8"

m1(5).values = 1
m1(5).frozen = True
m1(8).values = 0.7
m1(9).values = 0.3

m1(10).values = 10
m1(11).values = 0.5



Fit statistic  : Chi-Squared              2.591123e+09     using 1384 bins.

Test statistic : Chi-Squared              2.591123e+09     using 1384 bins.
 Null hypothesis probability of 0.000000e+00 with 1376 degrees of freedom
 Current data and model not fit yet.
  parameter 1 is not frozen.

Fit statistic  : Chi-Squared              2.591123e+09     using 1384 bins.

Test statistic : Chi-Squared              2.591123e+09     using 1384 bins.
 Null hypothesis probability of 0.000000e+00 with 1376 degrees of freedom
 Current data and model not fit yet.

Fit statistic  : Chi-Squared              2.645048e+09     using 1384 bins.

Test statistic : Chi-Squared              2.645048e+09     using 1384 bins.
 Null hypothesis probability of 0.000000e+00 with 1376 degrees of freedom
 Current data and model not fit yet.
  parameter 3 is not frozen.

Fit statistic  : Chi-Squared              5.515619e+10     using 1384 bins.

Test statistic : Chi-Squared              5.515619e+10     using 1384

In [6]:
x.Fit.statMethod = "chi"
x.Fit.query = "yes"
x.Fit.perform()

Default fit statistic is set to: Chi-Squared
   This will apply to all current and newly loaded spectra.

Fit statistic  : Chi-Squared              5.227588e+09     using 1384 bins.

Test statistic : Chi-Squared              5.227588e+09     using 1384 bins.
 Null hypothesis probability of 0.000000e+00 with 1376 degrees of freedom
 Current data and model not fit yet.
                                   Parameters
Chi-Squared  |beta|/N    Lvl          1:nH       2:Gamma        3:kT_e        7:norm         8:Tin        9:norm         10:kT       11:norm
1752.96      15266.8      -3      0.282498       1.69722       2.82713    0.00349531      0.800976     0.0489594       5.12909   0.000929754
1558.95      113818       -3      0.298202       1.70201       4.47517    0.00348478      0.789880     0.0736142      0.762297   0.000393203
1557.56      7668.46       0      0.298118       1.70434       4.58301    0.00348254      0.788524     0.0742847      0.714109    0.00201310
1557.52      68.0195

In [7]:
# --- NEATLY PRINT BEST FIT OUTPUT ---
print("\n" + "="*95)
print(f"{'GROUP':<5} | {'C#':<3} | {'P#':<3} | {'COMPONENT':<12} | {'PARAMETER':<12} | {'VALUE':<12} | {'ERROR'}")
print("-" * 95)

# Track the global parameter index manually
global_par_idx = 1

for comp_idx, comp_name in enumerate(m1.componentNames, start=1):
    comp = getattr(m1, comp_name)
    for par_name in comp.parameterNames:
        par = getattr(comp, par_name)

        # Formatting Value and Error string
        val = par.values[0]
        if par.link != "":
            err_str = f"LINKED ({par.link})"
        elif par.frozen:
            err_str = "FROZEN"
        else:
            err_str = f"+/- {par.sigma:.5f}" if par.sigma > 0 else "N/A"

        # Print row (Group is always 1 for single dataset)
        print(f"{'1':<5} | {comp_idx:<3} | {global_par_idx:<3} | {comp_name:<12} | {par_name:<12} | {val:<12.5f} | {err_str}")

        global_par_idx += 1

print("-" * 95)
print(f"Final {x.Fit.statMethod.upper()}: {x.Fit.statistic:.2f}")
print(f"Degrees of Freedom: {x.Fit.dof}")
print(f"Reduced Statistic: {x.Fit.statistic/x.Fit.dof:.3f}")
print("="*95 + "\n")


GROUP | C#  | P#  | COMPONENT    | PARAMETER    | VALUE        | ERROR
-----------------------------------------------------------------------------------------------
1     | 1   | 1   | TBabs        | nH           | 0.29764      | +/- 0.06852
1     | 2   | 2   | nthComp      | Gamma        | 1.71030      | +/- 17.56360
1     | 2   | 3   | nthComp      | kT_e         | 4.08253      | +/- 128.28945
1     | 2   | 4   | nthComp      | kT_bb        | 0.78602      | LINKED (= p8)
1     | 2   | 5   | nthComp      | inp_type     | 1.00000      | FROZEN
1     | 2   | 6   | nthComp      | Redshift     | 0.00000      | FROZEN
1     | 2   | 7   | nthComp      | norm         | 0.00348      | +/- 0.12391
1     | 3   | 8   | diskbb       | Tin          | 0.78602      | +/- 3.61711
1     | 3   | 9   | diskbb       | norm         | 0.07393      | +/- 57.30793
1     | 4   | 10  | bbodyrad     | kT           | 0.96798      | +/- 316.84730
1     | 4   | 11  | bbodyrad     | norm         | 0.00595      |

In [8]:
# --- 5. PLOTTING ---

# Set rebinning for the plot only (e.g., min significance 7, max 10 bins)
x.Plot.setRebin(3, 3)
x.Plot.device = "/null"
x.Plot.xAxis = "keV"
x.Plot.add = True

# Extract Data
x.Plot("eeuf")
energy_x   = x.Plot.x(1)
energy_err = x.Plot.xErr(1)
data_y     = x.Plot.y(1)
data_err   = x.Plot.yErr(1)
model_y    = x.Plot.model(1)


y_units = x.Plot.labels()[1]
print(y_units)
# Extract Residuals
x.Plot("delchi")
residuals  = x.Plot.y(1)

# Convert to numpy arrays - using raw data without masking zero-count bins
e  = np.array(energy_x)
ee = np.array(energy_err)
dy = np.array(data_y)
de = np.array(data_err)
my = np.array(model_y)
res = np.array(residuals)

#background

x.Plot("background")
bkg = x.Plot.y(1)

keV$^{2}$ (Photons cm$^{-2}$ s$^{-1}$ keV$^{-1}$)


In [9]:
# Plotting

%matplotlib notebook
plt.style.use('default')

plt.figure(figsize=(16, 9), dpi=100)
gs = plt.GridSpec(2, 1, height_ratios=[3, 1], hspace=0)

# Top panel: Data and Model
ax1 = plt.subplot(gs[0])
ax1.errorbar(e, dy, xerr=ee, yerr=de, fmt='o', ms=2, elinewidth=0.8, label='PN Data')
ax1.step(e, my, where='mid', color='C1', lw=1.5, zorder=10, label='Model')
ax1.set_yscale("log")
ax1.set_ylabel(fr"{y_units}", fontsize=16)
ax1.set_title(f"PN Spectrum: {m1.expression}")

# ax1.step(e, bkg, where='mid', color='black', lw=1.2, label='Background')
# Only hide the labels, keep the tick marks


# Bottom panel: Residuals
ax2 = plt.subplot(gs[1], sharex=ax1)
ax2.step(e, res, where='mid', color='black', lw=1.2)
# ax2.errorbar(e, res, xerr=ee, fmt='none', ecolor='black', elinewidth=0.8, capsize=2)
ax2.axhline(0, color='red', linestyle='--')
# ax2.set_ylim(-4,4)

ax2.set_xlabel("Energy (keV)",fontsize=16)
ax2.set_ylabel(r"Residuals ($\chi$)", fontsize=16)
ax1.legend(fontsize=14, loc='upper left')
plt.tight_layout()
plt.savefig(f'../products/0865600201/pn/spec/pn_{selected_interval}_fit.png', dpi=300, bbox_inches='tight')

<IPython.core.display.Javascript object>

In [10]:
# Quick check: Calculate counts from rates to decide on statistic
exposure = s.exposure
counts = np.array([r * exposure for r in s.values if r > 0])
min_counts = np.min(counts)
median_counts = np.median(counts)

print(f"Exposure: {exposure:.1f} s")
print(f"Min counts/bin: {min_counts:.1f}")
print(f"Median counts/bin: {median_counts:.1f}")

if min_counts < 20:
    print("\n--- RECOMMENDATION ---")
    print(f"Minimum counts ({min_counts:.1f}) is below 20.")
    print("Gaussian statistics (Chi-Squared) are invalid for low-count bins.")
    print("Use 'cstat' for the most reliable results.")
else:
    print("\n--- RECOMMENDATION ---")
    print(f"All bins have >= 20 counts (Min: {min_counts:.1f}).")
    print("You can safely use 'chi' (Chi-Squared) or 'cstat'.")

Exposure: 81275.1 s
Min counts/bin: 14.0
Median counts/bin: 90.0

--- RECOMMENDATION ---
Minimum counts (14.0) is below 20.
Gaussian statistics (Chi-Squared) are invalid for low-count bins.
Use 'cstat' for the most reliable results.
